In [1]:
import pandas as pd

In [6]:
data = pd.read_csv("data.xls", index_col=0)

In [7]:
data["team"].value_counts()

team
Nottingham Forest           70
Aston Villa                 70
Brentford                   70
West Ham United             70
Fulham                      70
Arsenal                     69
Wolverhampton Wanderers     69
Everton                     69
Crystal Palace              69
Liverpool                   69
Bournemouth                 69
Brighton and Hove Albion    69
Newcastle United            69
Manchester United           69
Tottenham Hotspur           69
Manchester City             69
Chelsea                     68
Leicester City              38
Leeds United                38
Southampton                 38
Luton Town                  32
Burnley                     32
Sheffield United            31
Name: count, dtype: int64

In [8]:
data["date"] = pd.to_datetime(data["date"])

In [9]:
data["venue_code"] = data["venue"].astype("category").cat.codes

In [10]:
data["opponent_code"] = data["opponent"].astype("category").cat.codes

In [11]:
data["hour_match"] = data["time"].str.replace(":.+","",regex=True).astype("int")

In [13]:
data["day_code"] = data["date"].dt.dayofweek

In [22]:
data["target"] = (data["result"] == "W").astype("int")

In [23]:
data

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opponent_code,hour_match,day_code,target
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,0.0,0,0,2024,Arsenal,1,17,12,5,1
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0.0,1,1,2024,Arsenal,0,7,20,0,1
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,0.0,1,1,2024,Arsenal,1,9,15,5,0
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,0.0,0,0,2024,Arsenal,1,15,16,6,1
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,0.0,0,0,2024,Arsenal,0,8,16,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1.0,3.0,Newcastle Utd,...,0.0,0,0,2023,Southampton,0,16,14,6,0
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3.0,4.0,Nott'ham Forest,...,0.0,1,1,2023,Southampton,0,17,20,0,0
44,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0.0,2.0,Fulham,...,0.0,0,0,2023,Southampton,1,9,15,5,0
45,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1.0,3.0,Brighton,...,1.0,0,0,2023,Southampton,0,4,14,6,0


In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
random = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [26]:
data_train = data[data['date'] < '2024-01-01']

In [27]:
data_test = data[data['date'] > '2024-01-01']

In [28]:
predictors = ["venue_code", "opponent_code", "hour_match", "day_code"]

In [29]:
random.fit(data_train[predictors], data_train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [30]:
prediction = random.predict(data_test[predictors])

In [31]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy = accuracy_score(data_test["target"], prediction)

In [34]:
accuracy

0.6163793103448276

In [35]:
merge = pd.DataFrame(dict(actual=data_test['target'], prediction=prediction))

In [36]:
pd.crosstab(index=merge["actual"], columns=merge["prediction"])

prediction,0,1
actual,,
0,110,41
1,48,33


In [37]:
from sklearn.metrics import precision_score

In [38]:
precision_score(data_test['target'], prediction)

0.44594594594594594